In [1]:
import numpy as np
import sys
sys.path.append('../../../codes/emission_line_codes/')
from importing_ews import *


In [2]:
lines = ["OII_DOUBLET", "HGAMMA", "HBETA", "OIII_4959", "OIII_5007", "NII_6548",\
         "HALPHA", "NII_6584", "SII_6716", "SII_6731"]



N_cv = 10
l = 6
run = 5
sv = '3'
scaling = 'sinh'

data = ['raw_masked']
#data = ['fastphot','raw_masked']
m = [81]
m_names = ['jax']
data_stats = np.zeros([3, len(data), len(m), len(lines)])

N = 13
path_to_codes = r"/Users/ASK126/Desktop/research/codes/emission_line_codes/"

EW_fit_all, EW_obs_all, y_fit_all, ivar_all, data_stats, _ = import_ews(N_bins=N, n = None, lines = lines, data = data, m = m, run = run, sv = sv, scaling = scaling, path_to_codes=path_to_codes, frac=1)

target_ids = []
redshifts = []
b = []
n_pre = []
for l in range(len(lines)):
    target_ids.append(np.load(path_to_codes + "/target_selection/zero_fluxes_sv" + sv + "_TARGETID_selection" + str(run) +\
                                  "_" + str(lines[l]) + '_bins' + str(N) + ".txt.npz")["arr_0"])
    target_ids_pre = np.load(path_to_codes + "/target_selection/sv" + sv + "_TARGETID_selection" + str(run) +\
                                  "_" + str(lines[l]) + ".txt.npz")["arr_0"]
    n_pre.append(len(target_ids_pre))
    sorter = np.argsort(target_ids_pre)
    b.append(sorter[np.searchsorted(target_ids_pre, target_ids[l], sorter=sorter)])
    redshifts.append(np.load(path_to_codes + "/target_selection/sv" + sv + "_Z_selection" + str(run) +\
                                    "_" + str(lines[l]) + ".txt.npz")["arr_0"][b[l]])

/Users/ASK126/Desktop/research/git_repos/emission-lines/notebooks/../../../codes/emission_line_codes/importing_ews.py:15: RuntimeWarning: invalid value encountered in log10
  fit = np.log10(fit)
/Users/ASK126/Desktop/research/git_repos/emission-lines/notebooks/../../../codes/emission_line_codes/importing_ews.py:17: RuntimeWarning: overflow encountered in power
  fit = 10**fit


In [4]:
for l in range(len(lines)):
    np.savez_compressed('../data/ews/sv3_obs_ews_'+lines[l], EW_obs_all[l][:,0,0])
    np.savez_compressed('../data/ews/sv3_fit_ews_'+lines[l], EW_fit_all[l][:,0,0])
    np.savez_compressed('../data/ews/sv3_ew_ivars_'+lines[l], ivar_all[l][:,0,0])
    np.savez_compressed('../data/target_ids/sv3_target_ids_'+lines[l], target_ids[l])
    np.savez_compressed('../data/redshifts/sv3_redshifts_'+lines[l], redshifts[l])

for l in range(len(lines)):
    number_of_decades = int(np.floor(n_pre[l]/10_000)) + 1
    fluxes = []
    for i in range(number_of_decades):
        fluxes.append(np.load('../data/fluxes/sv3_fluxes_raw_masked'+str(i)+'_selection'+str(run)+'_'+lines[l]+'_bins'+str(N)+'.txt.npz')['arr_0'])
    fluxes = np.concatenate(fluxes)
    fluxes = fluxes[b[l],:]
    np.savez_compressed('../data/fluxes/sv3_fluxes_masked_'+lines[l], fluxes)